In [1]:
import requests
import parsel
import time
from datetime import datetime
import re
import json

### Requisito 1

In [2]:
def fetch(url):
    time.sleep(1)
    try:
        response = requests.get(url=url, timeout=3)
        if response.ok:
            return response.text
        else:
            return None
    except requests.exceptions.ReadTimeout:
        return None

### Requisito 2

In [3]:
def scrape_novidades(html_content):
    selector = parsel.Selector(text=html_content)
    res = selector.css('.tec--list .tec--card__title__link')
    return list(map(lambda x: x.attrib['href'], res))

In [4]:
request_text = fetch('https://www.tecmundo.com.br/novidades')

In [5]:
scrape_novidades(request_text)

['https://www.tecmundo.com.br/minha-serie/232437-the-office-atrizes-serie-comentam-famoso-episodio-parkour.htm',
 'https://www.tecmundo.com.br/mercado/232458-microsoft-5-empresas-bilionarias-games-conhecer.htm',
 'https://www.tecmundo.com.br/mercado/232417-epic-games-volta-brigar-apple-devido-monopolio-ios.htm',
 'https://www.tecmundo.com.br/voxel/232477-ps4-pior-melhor-segundo-critica.htm',
 'https://www.tecmundo.com.br/voxel/232398-the-last-of-us-2-joel-esconde-detalhe-legal-flashback-ellie.htm',
 'https://www.tecmundo.com.br/minha-serie/232431-and-just-like-that-serie-agrada-fas-sex-and-the-city-critica.htm',
 'https://www.tecmundo.com.br/produto/232484-this-is-us-saiba-onde-assistir-serie-dramatica-nbc.htm',
 'https://www.tecmundo.com.br/minha-serie/232000-7-servicos-voce-assistir-doramas-brasil.htm',
 'https://www.tecmundo.com.br/seguranca/232441-ataques-ciberneticos-aumentarao-eleicoes-2022-diz-especialista.htm',
 'https://www.tecmundo.com.br/minha-serie/232483-bel-air-reboot-um-

### Requisito 03

In [6]:
def scrape_next_page_link(html_content):
    selector = parsel.Selector(text=html_content)
    return selector.css('.tec--list a.tec--btn::attr("href")').get()

### Requisito 4

In [11]:
def extract_count(strings_list, regex_r):
    output = 0
    for y in strings_list:
        try:
            regex = re.search(regex_r, y)
            return int(regex.groups()[0])
        except AttributeError:
            pass
    return output


def parse_json_writer(jsons_str):
    output = None
    for json_str in jsons_str:
        if json_str is None:
            return None

        try:
            deserialized_json = json.loads(json_str)
            output = deserialized_json["author"]["name"]
        except KeyError:
            pass

    return output


def extract_writer(selector):
    selectors = [
        selector.css(".tec--author__info__link::text").get(),
        selector.css(
            ".z--flex tec--timestamp tec--timestamp__item::text"
        ).get(),
        parse_json_writer(
            selector.css('script[type="application/ld+json"]::text').getall()
        ),
    ]

    for writer in selectors:
        if writer is not None:
            return str.strip(writer)

    return None


def scrape_noticia(html_content):
    selector = parsel.Selector(text=html_content)

    seletected = selector.css(".tec--toolbar__item *::text").getall()

    output = {
        "url": selector.css('link[rel="canonical"]::attr(href)').get(),
        "title": selector.css("#js-article-title::text").get(),
        "timestamp": selector.css("#js-article-date::attr(datetime)").get(),
        "writer": extract_writer(selector),
        "shares_count": extract_count(seletected, r"(\d+) Compartilharam"),
        "comments_count": extract_count(seletected, r"(\d+) Comentários"),
        "summary": "".join(
            selector.css(".tec--article__body > p:first-child ::text").getall()
        ),
        "sources": list(
            map(
                str.strip,
                selector.css(".z--mb-16 div a::text").getall(),
            )
        ),
        "categories": list(
            map(str.strip, selector.css("#js-categories a::text").getall())
        ),
    }
    return output


In [12]:
scrape_noticia(
  fetch('https://www.tecmundo.com.br/minha-serie/215285-falcao-soldado-invernal-fas-reacao-curiosa-sebastian-stan.htm')
)

{'url': 'https://www.tecmundo.com.br/minha-serie/215285-falcao-soldado-invernal-fas-reacao-curiosa-sebastian-stan.htm',
 'title': 'Falcão e o Soldado Invernal: fãs têm reação curiosa ao ver Sebastian Stan',
 'timestamp': '2021-04-09T20:00:01',
 'writer': 'Gabriel Witiuk',
 'shares_count': 0,
 'comments_count': 2,
 'summary': 'Atualmente fazendo sucesso na série Falcão e o Soldado Invernal, Sebastian Stan tem interpretado Bucky Barnes desde o primeiro filme do Capitão América. Stan contou que os fãs da Marvel sempre tentam "ativar" o lado assassino do personagem quando reconhecem o ator nas ruas.',
 'sources': ['CBR', 'MCU Exchange'],
 'categories': ['Minha Série', 'Séries', 'Elenco', 'Marvel', 'Disney+']}